In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers

In [27]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = '/home/models/top_tuned/bottleneck_fc_model.h5'
train_data_dir = '/home/user/Документы/convnets/data/train'
validation_data_dir = '/home/user/Документы/convnets/data/validation'
nb_train_samples = 6500
nb_validation_samples = 2500
epochs = 20
batch_size = 20


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

In [25]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 5) + [1] * (nb_train_samples // 5) + [2] * (nb_train_samples // 5) + [3] * (nb_train_samples // 5) + [4] * (nb_train_samples // 5))
    train_labels = to_categorical(train_labels)
    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 5) + [1] * (nb_validation_samples // 5) + [2] * (nb_validation_samples // 5) + [3] * (nb_validation_samples // 5) + [4] * (nb_validation_samples // 5))
    validation_labels = to_categorical(validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    custom_adam = optimizers.Adam(lr = 0.00001)
    model.compile(optimizer=custom_adam,
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save(top_model_weights_path)

In [28]:
save_bottlebeck_features()

Found 6500 images belonging to 5 classes.
Found 2500 images belonging to 5 classes.


In [29]:
train_top_model()

Train on 6500 samples, validate on 2500 samples
Epoch 1/20
6500/6500 [==============================] - 4s 637us/sample - loss: 1.3227 - accuracy: 0.4655 - val_loss: 0.9034 - val_accuracy: 0.7388
Epoch 2/20
6500/6500 [==============================] - 3s 508us/sample - loss: 0.8722 - accuracy: 0.6957 - val_loss: 0.7241 - val_accuracy: 0.7852
Epoch 3/20
6500/6500 [==============================] - 4s 574us/sample - loss: 0.7171 - accuracy: 0.7546 - val_loss: 0.6509 - val_accuracy: 0.7996
Epoch 4/20
6500/6500 [==============================] - 3s 537us/sample - loss: 0.6344 - accuracy: 0.7840 - val_loss: 0.6112 - val_accuracy: 0.8136
Epoch 5/20
6500/6500 [==============================] - 4s 586us/sample - loss: 0.5819 - accuracy: 0.8031 - val_loss: 0.5852 - val_accuracy: 0.8184
Epoch 6/20
6500/6500 [==============================] - 3s 533us/sample - loss: 0.5434 - accuracy: 0.8191 - val_loss: 0.5669 - val_accuracy: 0.8288
Epoch 7/20
6500/6500 [==============================] - 4s 576us